In [3]:
import random
from datetime import datetime, timedelta
import pymysql
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# MySQL 연결
connection = pymysql.connect(
    host='database-1.cbcw28i2we7h.us-east-2.rds.amazonaws.com',
    user='kj',
    password='1234',
    database='nahonlab',
    charset='utf8mb4'
)
cursor = connection.cursor()

# 데이터 생성 설정
num_users = 30  # user_id 3부터 32까지
start_date = datetime(2024, 11, 1)
end_date = datetime(2024, 11, 30)

# 사용자별 초기 값 설정
initial_metrics = {
    user_id: {
        "height": random.randint(160, 190),  # 고정된 키 값
        "weight": random.uniform(50, 80),
        "fat_percent": random.uniform(15, 30),
        "muscle_mass": random.uniform(25, 40),
    }
    for user_id in range(3, 33)  # user_id 3부터 32까지
}

# 기본 값 생성 함수
def generate_metrics(user_id, prev_weight, prev_fat_percent, prev_muscle_mass):
    calories_intake = random.randint(1800, 3500)  # 섭취 칼로리
    calories_burned = random.randint(500, 1000)   # 소모 칼로리

    # 체중 변화: 섭취 > 소모 -> 증가, 섭취 < 소모 -> 감소
    calorie_diff = calories_intake - calories_burned
    weight_change = calorie_diff / 7000  # 1kg = 약 7000kcal
    new_weight = max(45, prev_weight + weight_change)  # 체중은 최소 45kg

    # 체지방률 변화: 섭취 > 소모 -> 증가, 운동량 높으면 감소
    fat_change = 0.2 if calorie_diff > 500 else (-0.1 if calories_burned > 800 else 0)
    new_fat_percent = max(5, min(40, prev_fat_percent + fat_change))  # 체지방률 5~40% 제한

    # 근육량 변화: 소모 칼로리가 높으면 증가, 섭취만 높으면 유지
    muscle_change = 0.1 if calories_burned > 700 else (-0.05 if calorie_diff > 0 else 0.02)
    new_muscle_mass = max(20, prev_muscle_mass + muscle_change)  # 근육량 최소 20kg

    return new_weight, new_fat_percent, new_muscle_mass, calories_burned, calories_intake

# 데이터 생성 및 삽입
for user_id in range(3, 33):
    user_metrics = initial_metrics[user_id]
    prev_weight = user_metrics["weight"]
    prev_fat_percent = user_metrics["fat_percent"]
    prev_muscle_mass = user_metrics["muscle_mass"]
    height = user_metrics["height"]  # 고정된 키 값

    date = start_date
    while date <= end_date:
        # 새로운 데이터 생성
        new_weight, new_fat_percent, new_muscle_mass, calories_burned, calories_intake = generate_metrics(
            user_id, prev_weight, prev_fat_percent, prev_muscle_mass
        )

        # 데이터 삽입
        cursor.execute(
            """
            INSERT INTO user_body_metrics (user_id, record_date, height, weight, fat_percent, muscle_mass, calories_burned, calories_intake)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (
                user_id,
                date.strftime('%Y-%m-%d'),
                height,  # 고정된 키 값 사용
                round(new_weight, 2),
                round(new_fat_percent, 2),
                round(new_muscle_mass, 2),
                calories_burned,
                calories_intake
            )
        )

        # 이전 값을 업데이트
        prev_weight = new_weight
        prev_fat_percent = new_fat_percent
        prev_muscle_mass = new_muscle_mass

        # 다음 날짜로 이동
        date += timedelta(days=1)

# 데이터베이스 커밋
connection.commit()

# 데이터 가져오기 및 상관관계 분석
query = """
SELECT weight, record_date, fat_percent, muscle_mass, calories_burned, calories_intake
FROM user_body_metrics
"""
data = pd.read_sql(query, connection)

# 상관관계 계산 및 시각화
correlation_matrix = data.corr()
print("Correlation Matrix:")
print(correlation_matrix)

sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix of User Body Metrics")
plt.show()

# 연결 닫기
cursor.close()
connection.close()

print("Data generation, insertion, and correlation analysis complete.")


In [1]:
import pandas as pd
import numpy as np
import pymysql
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import joblib

# MySQL 데이터베이스 연결 설정
connection = pymysql.connect(
    host='database-1.cbcw28i2we7h.us-east-2.rds.amazonaws.com',
    user='kj',
    password='1234',
    database='nahonlab',
    charset='utf8mb4'
)

# 데이터 불러오기
query = """
SELECT user_id, record_date, weight, fat_percent, muscle_mass, calories_burned, calories_intake
FROM user_body_metrics
"""
data = pd.read_sql(query, connection)
connection.close()

# 날짜 형식 변환 및 정렬
data['record_date'] = pd.to_datetime(data['record_date'])
data = data.sort_values(['user_id', 'record_date'])

# 피처와 타깃 변수 정의
features = data[['calories_burned', 'calories_intake']]
targets = data[['weight', 'fat_percent', 'muscle_mass']]

# 정규화
scaler_features = MinMaxScaler()
scaler_targets = MinMaxScaler()

if not features.empty:
    features_scaled = scaler_features.fit_transform(features)
    targets_scaled = scaler_targets.fit_transform(targets)
else:
    raise ValueError("데이터셋이 비어있습니다. 학습할 데이터가 필요합니다.")

# 시계열 데이터 생성 함수
def create_sequences(features, targets, time_steps=15):
    X, y = [], []
    for i in range(len(features) - time_steps):
        X.append(features[i:(i + time_steps)])
        y.append(targets[i + time_steps])
    return np.array(X), np.array(y)

# 시퀀스 데이터 생성
TIME_STEPS = 15
X, y = create_sequences(features_scaled, targets_scaled, TIME_STEPS)

# 학습 및 테스트 데이터 분리
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# 단일 모델 생성 및 학습
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=False, input_shape=input_shape))
    model.add(Dense(3))  # 예측할 타깃 변수의 수
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

model = create_lstm_model((X_train.shape[1], X_train.shape[2]))
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, shuffle=False)

# 모델 저장
model.save("global_model.h5")
joblib.dump(scaler_features, "global_scaler_features.pkl")
joblib.dump(scaler_targets, "global_scaler_targets.pkl")

print("전체 데이터를 사용하여 모델 학습 완료.")


2024-12-12 19:46:55.974226: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-12 19:46:55.974338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-12 19:46:56.006078: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-12 19:46:56.073776: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-12 19:46:57.168063: W tensorflow/compiler/tf2

Epoch 1/50


2024-12-12 19:47:05.170021: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-12-12 19:47:05.679427: I external/local_xla/xla/service/service.cc:168] XLA service 0x7e898c144e00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-12-12 19:47:05.679450: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2024-12-12 19:47:05.691488: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1734000425.874222    5918 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


28/36 [======================>.......] - ETA: 0s - loss: 0.1305

In [14]:
import joblib
import numpy as np
import tensorflow.keras.models as keras_models
import pandas as pd
import pymysql

# 시계열 데이터의 길이 (시간 단계)
TIME_STEPS = 15

# 예측 함수
def predict_for_user(user_id, predict_days=15):
    # MySQL 데이터베이스 연결 설정
    connection = pymysql.connect(
        host='database-1.cbcw28i2we7h.us-east-2.rds.amazonaws.com',
        user='kj',
        password='1234',
        database='nahonlab',
        charset='utf8mb4'
    )

    # 저장된 모델 및 스케일러 로드
    try:
        model = keras_models.load_model("global_model.h5")
        scaler_features = joblib.load("global_scaler_features.pkl")
        scaler_targets = joblib.load("global_scaler_targets.pkl")
    except FileNotFoundError:
        print("저장된 글로벌 모델 또는 스케일러가 없습니다.")
        connection.close()  # 연결 종료
        return

    # 데이터 불러오기
    try:
        data = pd.read_sql("""
        SELECT user_id, record_date, weight, fat_percent, muscle_mass, calories_burned, calories_intake
        FROM user_body_metrics
        """, connection)
    except Exception as e:
        print(f"데이터베이스 쿼리 실행 중 오류 발생: {e}")
        connection.close()  # 연결 종료
        return

    user_data = data[data['user_id'] == user_id].copy()
    features = user_data[['calories_burned', 'calories_intake']]

    if features.empty:
        print(f"User {user_id}에 대한 데이터가 없습니다.")
        connection.close()  # 연결 종료
        return

    features_scaled = scaler_features.transform(features)

    if len(features_scaled) < TIME_STEPS:
        print(f"User {user_id}에 대한 충분한 데이터가 없어 예측을 수행할 수 없습니다.")
        connection.close()  # 연결 종료
        return

    last_15_days_features = features_scaled[-TIME_STEPS:]
    current_sequence = last_15_days_features
    predictions_scaled = []

    for _ in range(predict_days):
        next_pred_scaled = model.predict(current_sequence[np.newaxis, :, :])[0]
        predictions_scaled.append(next_pred_scaled)
        next_feature = np.array([next_pred_scaled[0], next_pred_scaled[1]])
        current_sequence = np.vstack((current_sequence[1:], next_feature))

    predictions = scaler_targets.inverse_transform(predictions_scaled)

    # 연결 종료
    connection.close()
    return predictions

# 예측 결과 출력 함수
def display_predictions(user_id, predict_days=15):
    predictions = predict_for_user(user_id, predict_days)
    if predictions is not None:
        print(f"User {user_id}의 다음 {predict_days}일 예측 결과:")
        for day, pred in enumerate(predictions, 1):
            print(f"Day {day}: 체중={pred[0]:.2f}kg, 체지방률={pred[1]:.2f}%, 근육량={pred[2]:.2f}kg")

# 예시로 특정 사용자 예측 (예: 10일 예측)
display_predictions(user_id=8, predict_days=20)


2024-12-13 14:28:03.086151: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-13 14:28:03.086210: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-13 14:28:03.117520: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 14:28:03.183441: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-13 14:28:04.264303: W tensorflow/compiler/tf2

1/1 [==============================] - 0s 17ms/step


2024-12-13 14:28:09.018749: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


1/1 [==============================] - 0s 19ms/step
User 8의 다음 20일 예측 결과:
Day 1: 체중=75.69kg, 체지방률=25.52%, 근육량=34.97kg
Day 2: 체중=73.49kg, 체지방률=25.54%, 근육량=34.84kg
Day 3: 체중=72.49kg, 체지방률=25.72%, 근육량=34.92kg
Day 4: 체중=71.98kg, 체지방률=25.81%, 근육량=34.96kg
Day 5: 체중=71.46kg, 체지방률=25.90%, 근육량=34.99kg
Day 6: 체중=71.01kg, 체지방률=25.99%, 근육량=35.02kg
Day 7: 체중=71.05kg, 체지방률=26.00%, 근육량=35.02kg
Day 8: 체중=70.41kg, 체지방률=26.11%, 근육량=35.04kg
Day 9: 체중=68.98kg, 체지방률=26.34%, 근육량=35.09kg
Day 10: 체중=68.72kg, 체지방률=26.41%, 근육량=35.10kg
Day 11: 체중=70.38kg, 체지방률=26.16%, 근육량=35.04kg
Day 12: 체중=71.49kg, 체지방률=25.96%, 근육량=34.98kg
Day 13: 체중=70.39kg, 체지방률=26.10%, 근육량=35.00kg
Day 14: 체중=70.07kg, 체지방률=26.16%, 근육량=35.02kg
Day 15: 체중=69.66kg, 체지방률=26.22%, 근육량=35.03kg
Day 16: 체중=70.22kg, 체지방률=26.16%, 근육량=35.02kg
Day 17: 체중=70.56kg, 체지방률=26.09%, 근육량=35.00kg
Day 18: 체중=70.77kg, 체지방률=26.05%, 근육량=34.99kg
Day 19: 체중=70.89kg, 체지방률=26.03%, 근육량=34.99kg
Day 20: 체중=70.96kg, 체지방률=26.01%, 근육량=34.98kg
